# Build model from config in mmdete 

In [5]:
from   mmdet.models.builder import build_detector
from   mmcv import Config
import torch
import mmcv

In [10]:
!ls ./configs/yolo

__pycache__  yolov3_d53_320_273e_coco.py	  yolov3.py
README.md    yolov3_d53_mstrain-416_273e_coco.py  yolov5.py
yolov3_1.py  yolov3_d53_mstrain-608_273e_coco.py


In [11]:
from   tools.utils.config import  _get_config_directory
from   os.path import join,realpath,relpath
import glob
config_dpath = _get_config_directory()+"/yolo"
config_fpaths = list(glob.glob(join(config_dpath,'*.py')))
config_fpaths = [p for p in config_fpaths if p.find('_base_') == -1]
config_names = [relpath(p, config_dpath) for p in config_fpaths]

In [12]:
config_names

['yolov5.py',
 'yolov3_d53_mstrain-608_273e_coco.py',
 'yolov3_d53_320_273e_coco.py',
 'yolov3.py',
 'yolov3_1.py',
 'yolov3_d53_mstrain-416_273e_coco.py']

In [13]:
!cat ./configs/yolo/yolov5.py

_base_ = '../_base_/default_runtime.py'
# model settings
model = dict(
    type='YOLOV5',
    pretrained='',
    backbone=dict(type='CSPDarknet',gd=1.0,gw=1.0,out_layers=[9,6,4]),
    neck=dict(
        type='YOLOV3Neck',
        num_scales=3,
        in_channels=[1024, 512, 256],
        out_channels=[512, 256, 128]),
    bbox_head=dict(
        type='YOLOV3Head',
        num_classes=80,
        in_channels=[512, 256, 128],
        out_channels=[1024, 512, 256],
        loss_cls=dict(
            type='CrossEntropyLoss',
            use_sigmoid=True,
            loss_weight=1.0,
            reduction='sum')
            ))

# training and testing settings



### 1. Pares config yolov5.py 

In [16]:
for config_fname in config_names[0:1]:
    config_fpath = join(config_dpath, config_fname)
    config_mod = Config.fromfile(config_fpath)
print(config_fname,'\n',config_fpath,'\n',config_mod)

yolov5.py 
 /home/whf/Desktop/mmdetection/configs/yolo/yolov5.py 
 Config (path: /home/whf/Desktop/mmdetection/configs/yolo/yolov5.py): {'checkpoint_config': {'interval': 1}, 'log_config': {'interval': 50, 'hooks': [{'type': 'TextLoggerHook'}]}, 'dist_params': {'backend': 'nccl'}, 'log_level': 'INFO', 'load_from': None, 'resume_from': None, 'workflow': [('train', 1)], 'model': {'type': 'YOLOV5', 'pretrained': '', 'backbone': {'type': 'CSPDarknet', 'gd': 1.0, 'gw': 1.0, 'out_layers': [9, 6, 4]}, 'neck': {'type': 'YOLOV3Neck', 'num_scales': 3, 'in_channels': [1024, 512, 256], 'out_channels': [512, 256, 128]}, 'bbox_head': {'type': 'YOLOV3Head', 'num_classes': 80, 'in_channels': [512, 256, 128], 'out_channels': [1024, 512, 256], 'loss_cls': {'type': 'CrossEntropyLoss', 'use_sigmoid': True, 'loss_weight': 1.0, 'reduction': 'sum'}}}}


In [38]:
for k,v in  config_mod.items():
    print(k,":",v)

checkpoint_config : {'interval': 1}
log_config : {'interval': 50, 'hooks': [{'type': 'TextLoggerHook'}]}
dist_params : {'backend': 'nccl'}
log_level : INFO
load_from : None
resume_from : None
workflow : [('train', 1)]
model : {'type': 'YOLOV5', 'pretrained': '', 'backbone': {'type': 'CSPDarknet', 'gd': 1.0, 'gw': 1.0, 'out_layers': [9, 6, 4]}, 'neck': {'type': 'YOLOV3Neck', 'num_scales': 3, 'in_channels': [1024, 512, 256], 'out_channels': [512, 256, 128]}, 'bbox_head': {'type': 'YOLOV3Head', 'num_classes': 80, 'in_channels': [512, 256, 128], 'out_channels': [1024, 512, 256], 'loss_cls': {'type': 'CrossEntropyLoss', 'use_sigmoid': True, 'loss_weight': 1.0, 'reduction': 'sum'}}}


In [39]:
config_mod.model

{'type': 'YOLOV5',
 'pretrained': '',
 'backbone': {'type': 'CSPDarknet',
  'gd': 1.0,
  'gw': 1.0,
  'out_layers': [9, 6, 4]},
 'neck': {'type': 'YOLOV3Neck',
  'num_scales': 3,
  'in_channels': [1024, 512, 256],
  'out_channels': [512, 256, 128]},
 'bbox_head': {'type': 'YOLOV3Head',
  'num_classes': 80,
  'in_channels': [512, 256, 128],
  'out_channels': [1024, 512, 256],
  'loss_cls': {'type': 'CrossEntropyLoss',
   'use_sigmoid': True,
   'loss_weight': 1.0,
   'reduction': 'sum'}}}

In [40]:
if 'pretrained' in config_mod.model:
    config_mod.model['pretrained'] = None

### 2. Builder detector

In [43]:
from mmdet.models import build_detector
model=build_detector(config_mod.model)

In [44]:
print(model)

YOLOV5(
  (backbone): CSPDarknet(
    (model): Sequential(
      (0): Focus(
        (conv): Conv(
          (conv): Conv2d(12, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act): SiLU()
        )
      )
      (1): Conv(
        (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): SiLU()
      )
      (2): C3(
        (cv1): Conv(
          (conv): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act): SiLU()
        )
        (cv2): Conv(
          (conv): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runni

In [47]:
from mmdet.models import DETECTORS
DETECTORS.module_dict

{'SingleStageDetector': mmdet.models.detectors.single_stage.SingleStageDetector,
 'ATSS': mmdet.models.detectors.atss.ATSS,
 'TwoStageDetector': mmdet.models.detectors.two_stage.TwoStageDetector,
 'CascadeRCNN': mmdet.models.detectors.cascade_rcnn.CascadeRCNN,
 'CornerNet': mmdet.models.detectors.cornernet.CornerNet,
 'FastRCNN': mmdet.models.detectors.fast_rcnn.FastRCNN,
 'FasterRCNN': mmdet.models.detectors.faster_rcnn.FasterRCNN,
 'FCOS': mmdet.models.detectors.fcos.FCOS,
 'FOVEA': mmdet.models.detectors.fovea.FOVEA,
 'FSAF': mmdet.models.detectors.fsaf.FSAF,
 'GFL': mmdet.models.detectors.gfl.GFL,
 'GridRCNN': mmdet.models.detectors.grid_rcnn.GridRCNN,
 'HybridTaskCascade': mmdet.models.detectors.htc.HybridTaskCascade,
 'MaskRCNN': mmdet.models.detectors.mask_rcnn.MaskRCNN,
 'MaskScoringRCNN': mmdet.models.detectors.mask_scoring_rcnn.MaskScoringRCNN,
 'NASFCOS': mmdet.models.detectors.nasfcos.NASFCOS,
 'PAA': mmdet.models.detectors.paa.PAA,
 'PointRend': mmdet.models.detectors.poin

In [52]:
# model=build_detector({'type':'YOLOV5','backbone':{}, 'neck':{},'bbox_head':{}})

In [63]:
m=DETECTORS.get(config_mod.model['type'])
# print(m(backbone={},neck={},bbox_head={}))

### 2.1 Builder Backbone

In [53]:
from mmdet.models import build_backbone,BACKBONES
BACKBONES.module_dict

{'Darknet': mmdet.models.backbones.darknet.Darknet,
 'ResNet': mmdet.models.backbones.resnet.ResNet,
 'ResNetV1d': mmdet.models.backbones.resnet.ResNetV1d,
 'DetectoRS_ResNet': mmdet.models.backbones.detectors_resnet.DetectoRS_ResNet,
 'DetectoRS_ResNeXt': mmdet.models.backbones.detectors_resnext.DetectoRS_ResNeXt,
 'HourglassNet': mmdet.models.backbones.hourglass.HourglassNet,
 'HRNet': mmdet.models.backbones.hrnet.HRNet,
 'ResNeXt': mmdet.models.backbones.resnext.ResNeXt,
 'RegNet': mmdet.models.backbones.regnet.RegNet,
 'Res2Net': mmdet.models.backbones.res2net.Res2Net,
 'SSDVGG': mmdet.models.backbones.ssd_vgg.SSDVGG,
 'CSPDarknet': mmdet.models.backbones.cspdarknet.CSPDarknet,
 'VGG16': mmdet.models.backbones.VGG16,
 'VGG9': mmdet.models.backbones.VGG9}

2.1.1 Use  backbone  builder

In [68]:
backbone_config=config_mod.model['backbone']
print(backbone_config)

{'type': 'CSPDarknet', 'gd': 1.0, 'gw': 1.0, 'out_layers': [9, 6, 4]}


In [67]:
backone=build_backbone(config_mod.model['backbone'])
print(backone)

CSPDarknet(
  (model): Sequential(
    (0): Focus(
      (conv): Conv(
        (conv): Conv2d(12, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): SiLU()
      )
    )
    (1): Conv(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): SiLU()
    )
    (2): C3(
      (cv1): Conv(
        (conv): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): SiLU()
      )
      (cv2): Conv(
        (conv): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): SiLU()
      )
      (cv3): Conv(
  

2.1.2 Use code hook

In [71]:
bo=BACKBONES.get(backbone_config['type'])
bc=backbone_config.pop('type')
print(bc)

CSPDarknet


In [72]:
backbone_config

{'gd': 1.0, 'gw': 1.0, 'out_layers': [9, 6, 4]}

In [75]:
bo(**backbone_config)

CSPDarknet(
  (model): Sequential(
    (0): Focus(
      (conv): Conv(
        (conv): Conv2d(12, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): SiLU()
      )
    )
    (1): Conv(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): SiLU()
    )
    (2): C3(
      (cv1): Conv(
        (conv): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): SiLU()
      )
      (cv2): Conv(
        (conv): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): SiLU()
      )
      (cv3): Conv(
  

### 2.2 Builder  necks 

In [78]:
from mmdet.models import build_neck,NECKS
NECKS.module_dict

{'BFP': mmdet.models.necks.bfp.BFP,
 'FPN': mmdet.models.necks.fpn.FPN,
 'FPN_CARAFE': mmdet.models.necks.fpn_carafe.FPN_CARAFE,
 'HRFPN': mmdet.models.necks.hrfpn.HRFPN,
 'NASFPN': mmdet.models.necks.nas_fpn.NASFPN,
 'NASFCOS_FPN': mmdet.models.necks.nasfcos_fpn.NASFCOS_FPN,
 'PAFPN': mmdet.models.necks.pafpn.PAFPN,
 'RFP': mmdet.models.necks.rfp.RFP,
 'YOLOV3Neck': mmdet.models.necks.yolo_neck.YOLOV3Neck}